In [2]:

import tkinter as tk
from tkinter import filedialog
import csv
import requests
import concurrent.futures



root = tk.Tk()
root.withdraw()

file_path = filedialog.askopenfilename(filetypes=[("Text Files", "*.txt")])

# Open the CSV file and create a list of dictionaries
with open(file_path, "r", encoding="utf-16-le", newline="\r\n") as file:
    reader = csv.DictReader(file, delimiter="\t")
    fieldnames = reader.fieldnames
    fieldnames.append("response")
    rows = list(reader)
    chunk_size = 5000
    chunks = [rows[i:i+chunk_size] for i in range(0, len(rows), chunk_size)]

def send_request(row):
    # Get the URL from the "Photo URL" field
    url = row["Photo URL"]
    if url == '': return row
    session = requests.Session()
    a = requests.adapters.HTTPAdapter(max_retries=0)
    b = requests.adapters.HTTPAdapter(max_retries=0)
    session.mount('http://', a)
    session.mount('https://', b)
    
    # Send a GET request to the URL
    try: 
        response = session.get(url, timeout=5)
        row["response"] = response.status_code
    except Exception as e: row["response"] = str(e)
    return row    
# Use a ThreadPoolExecutor to execute the requests in parallel
for i, chunk in enumerate(chunks):
    with concurrent.futures.ThreadPoolExecutor(max_workers=50) as executor:
        results = executor.map(send_request, chunk)

    # Write the updated rows back to the CSV file
    with open(f'/home/tnmon/git/vlurl/results/final-results-part-{i}appended.txt', "w", encoding="utf-16-le") as file:
        writer = csv.DictWriter(file, fieldnames=fieldnames, delimiter="\t")
        writer.writeheader()
        writer.writerows(results)